In [1]:
# ==============================================================================

# 👩‍💻 Author    : Hyelim Jo (Adapted by Gemini AI)
# 🎯 Purpose   : AI 윤리성 리스크 진단 에이전트 v1.0 - Risk Assessor
# 📅 Created   : 2025-10-23
# 📜 Note      : risk_assessor.ipynb

# ==============================================================================

In [2]:
# -------------------------------- Update Log ----------------------------------

# 2025-10-23 09:17 / 초기 생성 / Evidence Collector의 출력을 입력으로 받도록 구조 설계

# ------------------------------------------------------------------------------

In [3]:
# step1. 라이브러리 불러오기
import os
import json
from typing import Dict, List, Any
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
# Pydantic을 사용하기 위해 langchain_core에서 Base Model 임포트 (langchain v0.2.x 이상 권장)
from langchain_core.pydantic_v1 import BaseModel, Field 
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

print("✅ 라이브러리 불러오기 완료!")

✅ 라이브러리 불러오기 완료!


c:\Users\SKAX\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-JIaWGMA_-py3.11\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# step2. LLM 초기화 및 설정 (LLM_MODEL 및 JSON_SCHEMA는 기존과 동일)
try:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2) 
    print("✅ ChatOpenAI LLM 초기화 완료!")
except Exception as e:
    print(f"⚠️ ChatOpenAI 초기화 실패: {e}")
    llm = None
    
ASSESSOR_OUTPUT_SCHEMA = {
    "category": "string",
    "risk_level": "string (High, Limited, Minimal 중 택 1)",
    "assessment_summary": "string (평가 근거 및 핵심 이슈를 한국어로 3줄 이내 요약)",
    "recommendation_focus": "string (Mitigation 에이전트가 집중해야 할 구체적인 개선 방향 (예: '데이터 다양성 확보', '알고리즘 설명가능성 강화'))"
}

✅ ChatOpenAI LLM 초기화 완료!


In [5]:
# step3. 리스크 평가 프롬프트 정의 (기존과 동일)
ASSESSMENT_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", 
         "당신은 AI 윤리 리스크를 평가하는 전문가입니다. "
         "EU AI Act, OECD, UNESCO와 같은 **Baseline 근거**와 **최신 사회 이슈 근거**의 요약을 종합하여, "
         "제공된 서비스에 대해 특정 리스크 카테고리(편향성, 프라이버시, 투명성 등)의 위험도를 판단하고, 결과를 JSON 형식으로만 반환하세요. "
         "위험도는 **High, Limited, Minimal** 중 하나여야 합니다."
         "\n\n[출력 JSON 스키마]:\n{schema}"
        ),
        ("human", 
         "--- 서비스 및 리스크 정보 ---"
         "\n서비스명: {service_name}"
         "\n평가 리스크 카테고리: {category}"
         "\n--- Baseline 근거 요약 ---"
         "\n{baseline_summaries}"
         "\n--- Issue 근거 요약 ---"
         "\n{issue_summaries}"
         "\n\n위 정보를 바탕으로 {service_name}의 {category} 리스크 수준을 평가하고, 개선 권고안의 초점을 설정하세요."
        )
    ]
)

In [6]:
# step4. 리스크 평가 함수 정의 (State를 입력으로 받도록 변경)
def assess_risk_and_update_state(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    State에서 증거 데이터를 읽고 리스크를 평가한 후, State에 결과를 저장합니다.
    """
    if not llm:
        print("❌ LLM 초기화 실패로 평가를 수행할 수 없습니다.")
        state['assessment_status'] = "Error: LLM failed to initialize."
        return state
    
    # 💡 State에서 Evidence Collector의 결과 읽기
    evidence_data = state.get('evidence_data', {})
    if not evidence_data:
        print("❌ State에 'evidence_data'가 없습니다. 평가를 건너뜁니다.")
        state['assessment_status'] = "Error: Missing evidence_data in state."
        return state
    
    service_name = evidence_data.get('query', 'AI 서비스')
    risk_categories = list(evidence_data['scores'].keys())
    
    assessed_risks = []
    parser = JsonOutputParser(pydantic_object=None)
    
    print(f"\n⚖️ Risk Assessor 시작: {service_name}의 리스크 평가")
    
    for category in risk_categories:
        print(f"\n   🔎 {category.upper()} 리스크 평가 중...")
        
        # 1. Baseline 근거 요약 결합
        baseline_summaries = [
            f"- [Baseline] 출처: {src['source']} {src['chunk_info']}. 요약: {src['summary']}"
            for src in evidence_data.get('baseline_sources', []) if src.get('category') == category
        ]
        
        # 2. Issue 근거 요약 결합
        issue_summaries = [
            f"- [Issue] 출처: {src['source']}. 요약: {src['summary']}"
            for src in evidence_data.get('issue_sources', []) if src.get('category') == category
        ]
        
        baseline_text = "\n".join(baseline_summaries) if baseline_summaries else "증거 없음 (법적/윤리 기준 미확인 또는 무관)"
        issue_text = "\n".join(issue_summaries) if issue_summaries else "증거 없음 (최신 사회적 이슈 미발견)"

        chain = ASSESSMENT_PROMPT | llm | parser

        try:
            assessment_result = chain.invoke({
                "schema": json.dumps(ASSESSOR_OUTPUT_SCHEMA, indent=2, ensure_ascii=False),
                "service_name": service_name,
                "category": category,
                "baseline_summaries": baseline_text,
                "issue_summaries": issue_text
            })
            
            assessment_result['category'] = category 
            assessed_risks.append(assessment_result)
            print(f"     ✅ 평가 완료: {category.upper()} -> {assessment_result.get('risk_level', 'Unknown')}")
            
        except Exception as e:
            print(f"     ❌ 평가 실패 ({category}): {e}")
            assessed_risks.append({
                "category": category,
                "risk_level": "Error",
                "assessment_summary": f"평가 중 오류 발생: {e}",
                "recommendation_focus": "평가 실패"
            })
            
    # 최종 결과 구조화
    final_assessment = {
        "service_name": service_name,
        "assessed_risks": assessed_risks,
    }
    
    # 💡 State에 평가 결과 저장
    state['assessment_result'] = final_assessment
    state['assessment_status'] = "Success"
    
    print("\n✅ Risk Assessor 평가 및 State 업데이트 완료!")
    return state # 업데이트된 State 반환

In [7]:
# step5. 테스트 실행 (State 딕셔너리를 직접 생성하여 테스트)

print("\n" + "="*60)
print("⚖️ Risk Assessor 시작 (State 기반 테스트)...")
print("="*60)

# 💡 테스트를 위해 임시 State 객체 생성
# 이 데이터는 실제로는 service_profiler.ipynb, evidence_collector.ipynb 실행 후 state에 저장된 상태여야 함.
test_state = {
    # Service Profiler의 결과 (선택 사항)
    "service_profile": {
        "service_name": "이력서 분석 추천 시스템",
        "service_type": "recruitment system", 
        "risk_categories": ["bias", "privacy", "transparency"]
    },
    # Evidence Collector의 결과 (필수)
    "evidence_data": {
        "query": "이력서 분석 추천 시스템",
        "scores": {"bias": 1.0, "privacy": 1.0, "transparency": 1.0},
        "baseline_sources": [
            {"category": "bias", "source": "EU_AI_Act.pdf", "chunk_info": "(페이지 10의 내용)", "summary": "EU AI Act는 채용 AI를 고위험으로 분류하며, 기본권 침해 위험을 줄이기 위한 편향성 모니터링을 의무화한다.", "full_content": "..."},
            {"category": "privacy", "source": "OECD_Privacy_2024.pdf", "chunk_info": "(페이지 5의 내용)", "summary": "OECD는 민감 정보 처리 시 익명화 기술 적용과 데이터 주체의 통제권 강화를 권고한다.", "full_content": "..."}
        ],
        "issue_sources": [
            {"category": "bias", "source": "AI Ethics Today", "url": "...", "summary": "최근 기사에 따르면, AI 채용 시스템이 특정 집단에게 불리하게 작동하여 사회적 논란이 되고 있으며 공정성에 대한 의문이 제기되고 있다.", "full_content": "..."}
        ]
    }
}

# 리스크 평가 실행 (State 업데이트)
updated_state = assess_risk_and_update_state(test_state)

print("\n" + "="*60)
print("📢 Risk Assessor 최종 결과 (업데이트된 State 내용)")
print("="*60)
# 업데이트된 state['assessment_result'] 내용 출력
print(json.dumps(updated_state.get('assessment_result'), indent=2, ensure_ascii=False))

print(f"\n🔗 다음 단계: Mitigation Recommender는 state['assessment_result']를 입력으로 사용합니다.")
print("="*60)


⚖️ Risk Assessor 시작 (State 기반 테스트)...

⚖️ Risk Assessor 시작: 이력서 분석 추천 시스템의 리스크 평가

   🔎 BIAS 리스크 평가 중...
     ✅ 평가 완료: BIAS -> High

   🔎 PRIVACY 리스크 평가 중...
     ✅ 평가 완료: PRIVACY -> Limited

   🔎 TRANSPARENCY 리스크 평가 중...
     ✅ 평가 완료: TRANSPARENCY -> Minimal

✅ Risk Assessor 평가 및 State 업데이트 완료!

📢 Risk Assessor 최종 결과 (업데이트된 State 내용)
{
  "service_name": "이력서 분석 추천 시스템",
  "assessed_risks": [
    {
      "category": "bias",
      "risk_level": "High",
      "assessment_summary": "이력서 분석 추천 시스템은 EU AI Act에 따라 고위험으로 분류되며, 편향성 모니터링이 필수적입니다. 최근 AI 채용 시스템의 불공정성 문제로 사회적 논란이 발생하고 있어, 기본권 침해 위험이 큽니다.",
      "recommendation_focus": "데이터 다양성 확보"
    },
    {
      "category": "privacy",
      "risk_level": "Limited",
      "assessment_summary": "OECD는 민감 정보 처리 시 익명화 기술 적용과 데이터 주체의 통제권 강화를 권고하고 있으나, 최신 사회적 이슈는 발견되지 않았습니다. 따라서 프라이버시 리스크는 제한적입니다.",
      "recommendation_focus": "데이터 주체의 통제권 강화"
    },
    {
      "category": "transparency",
      "risk_level": "Minimal",
      "assessment_summary